Generate a map of a Surface Code, including Physical Qubits, Auxillary Ancillas, and bulk Ancillas.   
The map will be used later as input to generate stim circuit. 

In [1]:
import numpy as np

def generate_surface_code_matrix_with_shifts(n, m):
    rows = 2 * n + 1
    cols = 2 * m + 1
    
    # Initialize the matrix with '#' (representing empty spaces)
    matrix = np.full((rows, cols), '#', dtype=object)
    
    q_counter = 1  # Counter for qubits
    b_counter = 1  # Counter for bulk ancillas
    a_counter = 1  # Counter for auxiliary ancillas
    
    # Place physical qubits (Q)
    qubit_positions = {}
    for i in range(1, rows, 2):
        for j in range(1, cols, 2):
            matrix[i, j] = f'Q{q_counter}'
            qubit_positions[f'Q{q_counter}'] = (i, j)
            q_counter += 1
    
    # Place bulk ancillas (B)
    for i in range(2, rows-2, 2):
        for j in range(2, cols-2, 2):
            matrix[i, j] = f'B{b_counter}'
            b_counter += 1

    # Create the boundary qubit list
    boundary_qubits = []

    # Top row (left to right)
    for j in range(1, cols, 2):
        boundary_qubits.append(f'Q{matrix[1, j][1:]}')

    # Right column (top to bottom)
    for i in range(3, rows, 2):
        boundary_qubits.append(f'Q{matrix[i, cols - 2][1:]}')

    # Bottom row (right to left)
    for j in range(cols - 2, 0, -2):
        boundary_qubits.append(f'Q{matrix[rows - 2, j][1:]}')

    # Left column (bottom to top)
    for i in range(rows - 4, 0, -2):
        boundary_qubits.append(f'Q{matrix[i, 1][1:]}')

    # Remove duplicates from the boundary list
    boundary_qubits = list(dict.fromkeys(boundary_qubits))

    
    # Place auxiliary ancillas (A) between pairs of boundary qubits with shifts
    for k in range(1, len(boundary_qubits), 2):
        q1 = boundary_qubits[k - 1]
        q2 = boundary_qubits[k]

        # Get positions of q1 and q2
        q1_pos = qubit_positions[q1]
        q2_pos = qubit_positions[q2]
        
        if q1_pos[0] == q2_pos[0]:  # Same row
            ancilla_position = (q1_pos[0], (q1_pos[1] + q2_pos[1]) // 2)
            if ancilla_position[0] == 1:
                ancilla_position = (0, ancilla_position[1])  # Shift to top row
            elif ancilla_position[0] == rows - 2:
                ancilla_position = (rows - 1, ancilla_position[1])  # Shift to bottom row
        else:  # Same column
            ancilla_position = ((q1_pos[0] + q2_pos[0]) // 2, q1_pos[1])
            if ancilla_position[1] == 1:
                ancilla_position = (ancilla_position[0], 0)  # Shift to left column
            elif ancilla_position[1] == cols - 2:
                ancilla_position = (ancilla_position[0], cols - 1)  # Shift to right column

        # Place auxiliary ancilla (A)
        matrix[ancilla_position] = f'A{a_counter}'
        a_counter += 1
    
    return matrix

# Custom print function to display the matrix nicely
def print_matrix(matrix):
    for row in matrix:
        print('[' + ', '.join(map(lambda x: str(x).ljust(3), row)) + ']')

# Example usage for a 3x5 grid
n = 5
m = 5

# Generate and print the matrix with shifts
result = generate_surface_code_matrix_with_shifts(n, m)
print("\nGenerated Surface Code Matrix:")
print_matrix(result)



Generated Surface Code Matrix:
[#  , #  , A1 , #  , #  , #  , A2 , #  , #  , #  , #  ]
[#  , Q1 , #  , Q2 , #  , Q3 , #  , Q4 , #  , Q5 , #  ]
[#  , #  , B1 , #  , B2 , #  , B3 , #  , B4 , #  , A3 ]
[#  , Q6 , #  , Q7 , #  , Q8 , #  , Q9 , #  , Q10, #  ]
[A8 , #  , B5 , #  , B6 , #  , B7 , #  , B8 , #  , #  ]
[#  , Q11, #  , Q12, #  , Q13, #  , Q14, #  , Q15, #  ]
[#  , #  , B9 , #  , B10, #  , B11, #  , B12, #  , A4 ]
[#  , Q16, #  , Q17, #  , Q18, #  , Q19, #  , Q20, #  ]
[A7 , #  , B13, #  , B14, #  , B15, #  , B16, #  , #  ]
[#  , Q21, #  , Q22, #  , Q23, #  , Q24, #  , Q25, #  ]
[#  , #  , #  , #  , A6 , #  , #  , #  , A5 , #  , #  ]


The Next Cell Takes the output of the previous cell, and cuts it vertically where teleportation is supposed to take place.   
The number of Physical Qubits to be kept unchanged on the left side is chosen using parameter c 

In [2]:
import numpy as np

def make_vertical_cut(matrix, c, spacing):
    rows, cols = matrix.shape
    keep_columns = 2 * c
    if keep_columns >= cols:
        raise ValueError("Not enough columns in the matrix to make the cut.")

    # Create a new matrix with the additional columns
    new_matrix = np.full((rows, cols + spacing), '#', dtype=object)

    # Copy the initial unchanged part of the matrix
    for i in range(rows):
        new_matrix[i, :keep_columns] = matrix[i, :keep_columns]

    # Determine the column to modify and copy
    modify_column = keep_columns

    # Change IDs in the modify column to begin with 'T'
    for i in range(rows):
        if isinstance(matrix[i, modify_column], str) and matrix[i, modify_column][0] in {'A', 'B'}:
            new_matrix[i, modify_column] = 'T' + f'{i+1}' 
        else:
            new_matrix[i, modify_column] = matrix[i, modify_column]

    # Make 3 copies of the modify column
    for i in range(rows):
        if isinstance(matrix[i, modify_column], str) and matrix[i, modify_column][0] in {'A', 'B'}:
            og= new_matrix[i, modify_column]
            for k in range(2):
                new_matrix[i, modify_column + spacing -k] = og + f'{3-k}'
            new_matrix[i, modify_column] = new_matrix[i, modify_column] +'1'
    # Copy the rest of the matrix, adjusting for the added columns
    for i in range(rows):
        new_matrix[i, modify_column + spacing+1 :cols + spacing] = matrix[i, modify_column + 1:cols]

    return new_matrix

# Custom print function to display the matrix nicely
def print_matrix(matrix):
    for row in matrix:
        print('[' + ', '.join(map(lambda x: str(x).ljust(3), row)) + ']')

# Example usage
c = 1
n = 3
m = 3
spacing= 7
# Example matrix (replace this with your actual matrix)
initial_matrix = generate_surface_code_matrix_with_shifts(n, m)


print("\nInitial Surface Code Matrix:")
print_matrix(initial_matrix)

# Make the vertical cut and insert spacing
cut_matrix = make_vertical_cut(initial_matrix, c, spacing)
print("\nMatrix after Vertical Cut and Spacing:")
print_matrix(cut_matrix)



Initial Surface Code Matrix:
[#  , #  , A1 , #  , #  , #  , #  ]
[#  , Q1 , #  , Q2 , #  , Q3 , #  ]
[#  , #  , B1 , #  , B2 , #  , A2 ]
[#  , Q4 , #  , Q5 , #  , Q6 , #  ]
[A4 , #  , B3 , #  , B4 , #  , #  ]
[#  , Q7 , #  , Q8 , #  , Q9 , #  ]
[#  , #  , #  , #  , A3 , #  , #  ]

Matrix after Vertical Cut and Spacing:
[#  , #  , T11, #  , #  , #  , #  , #  , T12, T13, #  , #  , #  , #  ]
[#  , Q1 , #  , #  , #  , #  , #  , #  , #  , #  , Q2 , #  , Q3 , #  ]
[#  , #  , T31, #  , #  , #  , #  , #  , T32, T33, #  , B2 , #  , A2 ]
[#  , Q4 , #  , #  , #  , #  , #  , #  , #  , #  , Q5 , #  , Q6 , #  ]
[A4 , #  , T51, #  , #  , #  , #  , #  , T52, T53, #  , B4 , #  , #  ]
[#  , Q7 , #  , #  , #  , #  , #  , #  , #  , #  , Q8 , #  , Q9 , #  ]
[#  , #  , #  , #  , #  , #  , #  , #  , #  , #  , #  , A3 , #  , #  ]


This one Extracts useful info to be used later by our Circuit Generator.   
Basically lists of Qubits, Ancillas, Xtype, Ztype, Auxillary, Bulk, Teleport, etc. 

In [3]:
matrix= cut_matrix
rows, cols = matrix.shape
B_Ancillas= []
A_Ancillas=[]
T3_Ancillas= []
T2_Ancillas=[]
T1_Ancillas=[]
Z_Ancillas= []
X_Ancillas= []
Qubits= []
for i in range(rows):
    for j in range(cols):
        if isinstance(matrix[i,j], str) and matrix[i, j][0] in {'A'}: 
            A_Ancillas.append(matrix[i,j])
        if isinstance(matrix[i,j], str) and matrix[i, j][0] in {'B'}:
            B_Ancillas.append(matrix[i,j])  
        if isinstance(matrix[i,j], str) and matrix[i, j][0] in {'T'} and matrix[i,j][-1] == '3':
            T3_Ancillas.append(matrix[i,j])    
        if isinstance(matrix[i,j], str) and matrix[i, j][0] in {'T'} and matrix[i,j][-1] == '2':
            T2_Ancillas.append(matrix[i,j])    
        if isinstance(matrix[i,j], str) and matrix[i, j][0] in {'T'} and matrix[i,j][-1] == '1':
            T1_Ancillas.append(matrix[i,j])                      
        if isinstance(matrix[i,j], str) and matrix[i, j][0] in {'Q'}:
            Qubits.append(matrix[i,j])

Z_Ancillas=[]
X_Ancillas=[]
for i in range(2,rows):
        counter=((i/2) -1)%2
        for j in range(cols):
            if isinstance(matrix[i,j], str) and matrix[i, j][0] in {'B'}:
                if counter%2 ==0 :
                    Z_Ancillas.append(matrix[i,j])
                    counter+=1
                else:
                    X_Ancillas.append(matrix[i,j])
                    counter+=1
            elif isinstance(matrix[i,j], str) and matrix[i, j][0] in {'T'} and matrix[i,j][-1] == '1':
                if counter%2 ==0 :
                    Z_Ancillas.append(matrix[i,j])
                    counter+=1
                else:
                    X_Ancillas.append(matrix[i,j])
                    counter+=1
for i in (0,rows-1):
    for j in range(cols):
        if isinstance(matrix[i,j], str) and matrix[i, j][0] in {'A'}:
            X_Ancillas.append(matrix[i,j])
        elif isinstance(matrix[i,j], str) and matrix[i, j][0] in {'T'} and matrix[i,j][-1] == '1':
            X_Ancillas.append(matrix[i,j])

for i in range(0,rows):
        for j in (0,cols-1):
            if isinstance(matrix[i,j], str) and matrix[i, j][0] in {'A'}:
                Z_Ancillas.append(matrix[i,j])
            elif isinstance(matrix[i,j], str) and matrix[i, j][0] in {'T'} and matrix[i,j][-1] == '1':
                Z_Ancillas.append(matrix[i,j])

for element in X_Ancillas.copy():  # Use a copy to avoid modifying the list while iterating
    if element.startswith('T') and element.endswith('1'):
        # Create a new element with the last digit changed to '3'
        new_element = element[:-1] + '3'
        # Append the new element to X_Ancillas
        X_Ancillas.append(new_element)

for element in Z_Ancillas.copy():  # Use a copy to avoid modifying the list while iterating
    if element.startswith('T') and element.endswith('1'):
        # Create a new element with the last digit changed to '3'
        new_element = element[:-1] + '3'
        # Append the new element to X_Ancillas
        Z_Ancillas.append(new_element)

        
print(X_Ancillas)
print(Z_Ancillas)
print(B_Ancillas)
print(T3_Ancillas)
print(T2_Ancillas)
print(T1_Ancillas)
print(A_Ancillas)
print(Qubits)


['B2', 'T51', 'T11', 'A3', 'T53', 'T13']
['T31', 'B4', 'A2', 'A4', 'T33']
['B2', 'B4']
['T13', 'T33', 'T53']
['T12', 'T32', 'T52']
['T11', 'T31', 'T51']
['A2', 'A4', 'A3']
['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9']


This is Also for prepration. It Outputs a list of tuples, where each tuple have its first Entry as Ancilla, followed By its neighbors in order (North West, North East, South East, South West)

In [4]:
def find_diagonal_neighbors_with_type(matrix, X_Ancillas, Z_Ancillas):
    rows, cols = matrix.shape
    diagonal_neighbors = []

    for i in range(rows):
        for j in range(cols):
            if isinstance(matrix[i, j], str) and matrix[i, j][0] in {'A', 'B', 'T'}:
                neighbors = []
                # North-West
                if i > 0 and j > 0:
                    neighbors.append(matrix[i-1, j-1])
                else:
                    neighbors.append(None)
                
                # North-East
                if i > 0 and j < cols - 1:
                    neighbors.append(matrix[i-1, j+1])
                else:
                    neighbors.append(None)
                
                # South-East
                if i < rows - 1 and j < cols - 1:
                    neighbors.append(matrix[i+1, j+1])
                else:
                    neighbors.append(None)
                
                # South-West
                if i < rows - 1 and j > 0:
                    neighbors.append(matrix[i+1, j-1])
                else:
                    neighbors.append(None)
                
                # Determine type and create a tuple with the element and its neighbors
                ancilla_type = 'X' if matrix[i, j] in X_Ancillas else 'Z' if matrix[i, j] in Z_Ancillas else ''
                diagonal_neighbors.append((ancilla_type, matrix[i, j], *neighbors))
    
    return diagonal_neighbors

#Example usage with X_Ancillas and Z_Ancillas lists
diagonal_neighbors = find_diagonal_neighbors_with_type(matrix, X_Ancillas, Z_Ancillas)

# Print the results
for entry in diagonal_neighbors:
    print(entry)


('X', 'T11', None, None, '#', 'Q1')
('', 'T12', None, None, '#', '#')
('X', 'T13', None, None, 'Q2', '#')
('Z', 'T31', 'Q1', '#', '#', 'Q4')
('', 'T32', '#', '#', '#', '#')
('Z', 'T33', '#', 'Q2', 'Q5', '#')
('X', 'B2', 'Q2', 'Q3', 'Q6', 'Q5')
('Z', 'A2', 'Q3', None, None, 'Q6')
('Z', 'A4', None, 'Q4', 'Q7', None)
('X', 'T51', 'Q4', '#', '#', 'Q7')
('', 'T52', '#', '#', '#', '#')
('X', 'T53', '#', 'Q5', 'Q8', '#')
('Z', 'B4', 'Q5', 'Q6', 'Q9', 'Q8')
('X', 'A3', 'Q8', 'Q9', None, None)


In [ ]:
def initialize_circuit(cut_matrix, noise):
    # Initialize a new circuit
    circuit = stim.Circuit()

    # List to collect all qubit IDs for the reset operation
    qubit_ids = []

    # Iterate through the cut_matrix and append QUBIT_COORDS to the circuit
    rows, cols = cut_matrix.shape
    for i in range(rows):
        for j in range(cols):
            if isinstance(cut_matrix[i, j], str) and cut_matrix[i, j] != '#':
                qubit_id = cut_matrix[i, j]
                if qubit_id.startswith('A'):
                    qubit_id = '1' + qubit_id[1:]
                elif qubit_id.startswith('B'):
                    qubit_id = '2' + qubit_id[1:]
                elif qubit_id.startswith('T'):
                    qubit_id = '3' + qubit_id[1:]
                elif qubit_id.startswith('Q'):
                    qubit_id = qubit_id[1:]
                
                # Append QUBIT_COORDS to the circuit
                circuit += stim.Circuit(f"QUBIT_COORDS({i},{j}) {qubit_id}")
                
                # Collect qubit ID for reset operation
                qubit_ids.append(int(qubit_id))
    
    # Append R operation for each qubit
    circuit.append('R', qubit_ids)
    circuit.append('DEPOLARIZE1', qubit_ids, noise)
    circuit.append('TICK')

    # Append H gate to the elements of T2_Ancillas
    T2_Ancilla_ids = ['3' + t[1:] for t in T2_Ancillas]
    circuit.append("H", [int(q) for q in T2_Ancilla_ids])
    
    # Append CX gate between each T2 and its corresponding T3
    T3_Ancilla_ids = ['3' + t[1:] for t in T3_Ancillas]
    for t2, t3 in zip(T2_Ancilla_ids, T3_Ancilla_ids):
        circuit.append("CX", [int(t2), int(t3)])
    
    return circuit

# Example usage
cut_matrix = np.array([
    ['#', '#', 'A1', '#', '#', '#', 'T11', 'T12', 'T13', '#', '#', '#', '#', '#', '#', '#', '#', 'A3', '#', '#', '#', 'A4', '#', '#', '#', '#'],
    ['#', 'Q1', '#', 'Q2', '#', 'Q3', '#', '#', '#', '#', '#', '#', '#', '#', 'Q4', '#', 'Q5', '#', 'Q6', '#', 'Q7', '#', 'Q8', '#', 'Q9', '#'],
    ['#', '#', 'B1', '#', 'B2', '#', 'T31', 'T32', 'T33', '#', '#', '#', '#', '#', '#', 'B4', '#', 'B5', '#', 'B6', '#', 'B7', '#', 'B8', '#', 'A5'],
    ['#', 'Q10', '#', 'Q11', '#', 'Q12', '#', '#', '#', '#', '#', '#', '#', '#', 'Q13', '#', 'Q14', '#', 'Q15', '#', 'Q16', '#', 'Q17', '#', 'Q18', '#'],
    ['A12', '#', 'B9', '#', 'B10', '#', 'T51', 'T52', 'T53', '#', '#', '#', '#', '#', '#', 'B12', '#', 'B13', '#', 'B14', '#', 'B15', '#', 'B16', '#', '#'],
    ['#', 'Q19', '#', 'Q20', '#', 'Q21', '#', '#', '#', '#', '#', '#', '#', '#', 'Q22', '#', 'Q23', '#', 'Q24', '#', 'Q25', '#', 'Q26', '#', 'Q27', '#'],
    ['#', '#', 'B17', '#', 'B18', '#', 'T71', 'T72', 'T73', '#', '#', '#', '#', '#', '#', 'B20', '#', 'B21', '#', 'B22', '#', 'B23', '#', 'B24', '#', 'A6'],
    ['#', 'Q28', '#', 'Q29', '#', 'Q30', '#', '#', '#', '#', '#', '#', '#', '#', 'Q31', '#', 'Q32', '#', 'Q33', '#', 'Q34', '#', 'Q35', '#', 'Q36', '#'],
    ['A11', '#', 'B25', '#', 'B26', '#', 'T91', 'T92', 'T93', '#', '#', '#', '#', '#', '#', 'B28', '#', 'B29', '#', 'B30', '#', 'B31', '#', 'B32', '#', '#'],
    ['#', 'Q37', '#', 'Q38', '#', 'Q39', '#', '#', '#', '#', '#', '#', '#', '#', 'Q40', '#', 'Q41', '#', 'Q42', '#', 'Q43', '#', 'Q44', '#', 'Q45', '#'],
    ['#', '#', '#', '#', 'A10', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', 'A9', '#', '#', '#', 'A8', '#', '#', '#', 'A7', '#', '#']
], dtype=object)

T2_Ancillas = ['T21', 'T22', 'T23']  # Example list of T2_Ancillas
T3_Ancillas = ['T31', 'T32', 'T33']  # Example list of T3_Ancillas

circuit = initialize_circuit(cut_matrix, 0.01)
print(circuit)
